# Welcome to Battle of the Neighborhoods.

<img src="https://image.cnbcfm.com/api/v1/image/104642925-1564671243133preview.jpg?v=1566051866">
Image source - Google Images

### This notebook will be mainly used for capstone project for the course 'Applied Data Science Capstone' on Coursera

In [98]:
import numpy as np 

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium 


Libraries imported.


Using the **pandas.io.html** library to call the function **read_html** to scrape the table from the web.

In [37]:
from pandas.io.html import read_html

This **read_html** function returns a list, in which the first element is a pandas dataframe containing the web table.

In [47]:
wikiurl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wikitable = read_html(wikiurl, attrs={'class':'wikitable sortable'})
df = wikitable[0]

In [48]:
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [49]:
df.shape

(180, 3)

Removing rows where the Borough is 'Not assigned'.

In [50]:
df = df[df['Borough'] != 'Not assigned']
df.reset_index(inplace=True, drop=True)

## Problem 1 Solution

In [51]:
df.head(12)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [52]:
df.shape

(103, 3)

Using the **pgeocode** library to get the latitudes and longitudes for a given postal code.

For further reading about the  library go to the **[PGEOCODE Documentation](https://pypi.org/project/pgeocode/)**

In [66]:
import pgeocode
nomi = pgeocode.Nominatim('ca')
ll = nomi.query_postal_code(df['Postal code'].to_list())

In [ ]:
df['Latitude'] = ll['latitude']
df['Longitude'] = ll['longitude']

## Problem 2 Solution

In [72]:
df.head(12)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.6555,-79.3626
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.7223,-79.4504
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.6641,-79.3889
5,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
6,M1B,Scarborough,Malvern / Rouge,43.8113,-79.1930
7,M3B,North York,Don Mills,43.7450,-79.3590
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.7063,-79.3094
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


In [80]:
df.at[76, 'Latitude'] = 43.6370
df.at[76, 'Longitude'] = -79.6158

In [73]:
df.shape

(103, 5)

In [109]:
latitude = 43.6532
longitude = -79.3832


map_ca = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ca)  
    
map_ca

In [147]:
df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [155]:
toronto_df = df[(df['Borough'] == 'Downtown Toronto') | (df['Borough'] == 'East Toronto') | (df['Borough'] == 'West Toronto') | (df['Borough'] == 'Central Toronto') ]
toronto_df.reset_index(drop=True, inplace=True)
toronto_df.drop(index=38, inplace=True)

C:\Users\CHIRAG\Anaconda3\lib\site-packages\pandas\core\frame.py:4097: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [156]:
toronto_df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.6555,-79.3626
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941
5,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754
6,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860
7,M6G,Downtown Toronto,Christie,43.6683,-79.4205
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.6496,-79.3833
9,M6H,West Toronto,Dufferin / Dovercourt Village,43.6655,-79.4378


In [159]:
latitude_t = 43.6577
longitude_t = -79.4216


map_toronto = folium.Map(location=[latitude_t, longitude_t], zoom_start=12)


for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {},'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [161]:
CLIENT_ID = 'DKV3OU5NWJTJSDZXT2PI1IUGPYIWQEBGX4C3AFN20YHHAEA4' 
CLIENT_SECRET = '4XXVG2MPGCBCWK42NN1PB21QVRW1GOICYRNI42C03LBLJXO2' 
VERSION = '20180604'
LIMIT = 100
radius = 500

Lets create to function to get all Venues in a given Neighborhoods in Toronto, using FourSquare API

In [162]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            

        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']
        

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [163]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Roseda

In [164]:
toronto_venues.shape

(1521, 7)

In [165]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,Regent Park / Harbourfront,43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,Regent Park / Harbourfront,43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,Regent Park / Harbourfront,43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Regent Park / Harbourfront,43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [166]:
toronto_venues.groupby(by='Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,91,91,91,91,91,91
Brockton / Parkdale Village / Exhibition Place,39,39,39,39,39,39
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,60,60,60,60,60,60
Central Bay Street,59,59,59,59,59,59
Christie,13,13,13,13,13,13
Church and Wellesley,72,72,72,72,72,72
Commerce Court / Victoria Hotel,100,100,100,100,100,100
Davisville,25,25,25,25,25,25
Davisville North,6,6,6,6,6,6


In [180]:
toronto_venues['Venue Category'].unique()

array(['Coffee Shop', 'Bakery', 'Breakfast Spot', 'Spa',
       'Mexican Restaurant', 'Event Space', 'Yoga Studio', 'Theater',
       'Pub', 'Thai Restaurant', 'Restaurant', 'Italian Restaurant',
       'Distribution Center', 'Food Truck', 'Gym / Fitness Center',
       'Electronics Store', 'Beer Store', 'Health Food Store',
       'Dance Studio', 'Park', 'Sushi Restaurant', 'Beer Bar',
       'Bubble Tea Shop', 'Gastropub', 'Ramen Restaurant', 'Diner',
       'Creperie', 'Burrito Place', 'Arts & Crafts Store', 'Burger Joint',
       'Hobby Shop', 'Café', 'Martial Arts Dojo', 'Indian Restaurant',
       'College Theater', 'Gym', 'College Cafeteria',
       'General Entertainment', 'Clothing Store', 'Plaza', 'Tea Room',
       'Comic Shop', 'Music Venue', 'Art Gallery', 'Hotel',
       'Sandwich Place', 'Sporting Goods Shop', 'Shopping Mall',
       'Steakhouse', 'Japanese Restaurant', 'College Rec Center',
       'American Restaurant', 'Cosmetics Shop', 'Tanning Salon',
       'Fast Fo

Creating Categorical Variables for each Venue Category

In [190]:
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 


fixed_columns = ['Neighborhood'] + toronto_onehot.columns[:-1].drop('Neighborhood').to_list()
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint
0,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Regent Park / Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0

In [191]:
toronto_onehot.shape

(1521, 210)

In [192]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint
0,Berczy Park,0.000000,0.010989,0.021978,0.000000,0.000000,0.000000,0.010989,0.000000,0.010989,0.032967,0.000000,0.000000,0.000,0.010989,0.000000,0.032967,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.021978,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.054945,0.000000,0.000000,0.021978,0.000000,0.000000,0.010989,0.021978,0.076923,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010989,0.000000,0.010989,0.000000,0.000000,0.000,0.010989,0.000000,0.000000,0.000000,0.000000,0.021978,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,0.000000,0.00,0.00,0.000000,0.021978,0.010989,0.000000,0.010989,0.000000,0.000000,0.000000,0.010989,0.010989,0.010989,0.010989,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.010989,0.000000,0.000000,0.00,0.000000,0.00,0.010989,0.010989,0.010989,0.010989,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.054945,0.010989,0.000000,0.000000,0.00000

In [193]:
toronto_grouped.shape

(38, 210)

Print each neighborhood with top 5 common venues

In [194]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1                Café  0.05
2               Hotel  0.05
3  Seafood Restaurant  0.04
4  Italian Restaurant  0.03


----Brockton / Parkdale Village / Exhibition Place----
                    venue  freq
0             Coffee Shop  0.08
1                    Café  0.08
2          Breakfast Spot  0.05
3  Thrift / Vintage Store  0.05
4               Gift Shop  0.05


----CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport----
                venue  freq
0         Coffee Shop  0.08
1          Restaurant  0.07
2                Café  0.05
3  Italian Restaurant  0.05
4                 Bar  0.05


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0.24
1  Middle Eastern Restaurant  0.05
2            Bubble Tea Shop  0.03
3             Sandwich Place  0.03
4                 Restaurant  0.03


----Christie----
           ven

Function to sort venues in the descending order

In [195]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [227]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Seafood Restaurant,Beer Bar,Restaurant,Italian Restaurant,Japanese Restaurant,Bakery,Sporting Goods Shop
1,Brockton / Parkdale Village / Exhibition Place,Café,Coffee Shop,Breakfast Spot,Gift Shop,Thrift / Vintage Store,Dance Studio,Brewery,Sandwich Place,Restaurant,Chiropractor
2,CN Tower / King and Spadina / Railway Lands / ...,Coffee Shop,Restaurant,Bar,Café,Italian Restaurant,Park,Gym / Fitness Center,Bakery,Bank,Speakeasy
3,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Bubble Tea Shop,Café,Breakfast Spot,Sandwich Place,Italian Restaurant,Japanese Restaurant,Restaurant,Shopping Mall
4,Christie,Café,Grocery Store,Coffee Shop,Baby Store,Park,Candy Store,Athletics & Sports,Playground,Ethiopian Restaurant,Eastern European Restaurant


Clustering the Neighborhoods

In [228]:
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)


kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


kmeans.labels_[0:38] 

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 5, 1, 1, 3, 1, 1, 2, 1, 0, 0, 1,
       1, 1, 1, 0, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Adding the Labels

In [229]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(38) 

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.6555,-79.3626,1,Coffee Shop,Restaurant,Breakfast Spot,Health Food Store,Italian Restaurant,Beer Store,Pub,Distribution Center,Thai Restaurant,Theater
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.6641,-79.3889,1,Italian Restaurant,Sushi Restaurant,Gym,Coffee Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Restaurant,Ramen Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,1,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Hotel,Restaurant,Tea Room,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,1,Café,Coffee Shop,Seafood Restaurant,Italian Restaurant,Gastropub,Hotel,Cocktail Bar,American Restaurant,Restaurant,Farmers Market
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,1,Health Food Store,Trail,Cheese Shop,Pub,Bakery,Gastropub,Department Store,Deli / Bodega,Fast Food Restaurant,Farmers Market
5,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,1,Coffee Shop,Café,Hotel,Seafood Restaurant,Beer Bar,Restaurant,Italian Restaurant,Japanese Restaurant,Bakery,Sporting Goods Shop
6,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860,1,Coffee Shop,Middle Eastern Restaurant,Bubble Tea Shop,Café,Breakfast Spot,Sandwich Place,Italian Restaurant,Japanese Restaurant,Restaurant,Shopping Mall
7,M6G,Downtown Toronto,Christie,43.6683,-79.4205,0,Café,Grocery Store,Coffee Shop,Baby Store,Park,Candy Store,Athletics & Sports,Playground,Ethiopian Restaurant,Eastern European Restaurant
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.6496,-79.3833,1,Restaurant,Café,Coffee Shop,American Restaurant,Steakhouse,Hotel,Asian Restaurant,Japanese Restaurant,Gym,Thai Restaurant
9,M6H,West Toronto,Dufferin / Dovercourt Village,43.6655,-79.4378,1,Bakery,Park,Pet Store,Middle Eastern Restaurant,Café,Bar,Bank,Pharmacy,Gym / Fitness Center,Grocery Store


# Problem 3 Solution

Visualizing the Clusters

In [230]:
map_clusters = folium.Map(location=[latitude_t, longitude_t], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Lets Examine the Cluster with Maximum Frequency

In [231]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Restaurant,Breakfast Spot,Health Food Store,Italian Restaurant,Beer Store,Pub,Distribution Center,Thai Restaurant,Theater
1,Downtown Toronto,1,Italian Restaurant,Sushi Restaurant,Gym,Coffee Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Restaurant,Ramen Restaurant
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Hotel,Restaurant,Tea Room,Ramen Restaurant
3,Downtown Toronto,1,Café,Coffee Shop,Seafood Restaurant,Italian Restaurant,Gastropub,Hotel,Cocktail Bar,American Restaurant,Restaurant,Farmers Market
4,East Toronto,1,Health Food Store,Trail,Cheese Shop,Pub,Bakery,Gastropub,Department Store,Deli / Bodega,Fast Food Restaurant,Farmers Market
5,Downtown Toronto,1,Coffee Shop,Café,Hotel,Seafood Restaurant,Beer Bar,Restaurant,Italian Restaurant,Japanese Restaurant,Bakery,Sporting Goods Shop
6,Downtown Toronto,1,Coffee Shop,Middle Eastern Restaurant,Bubble Tea Shop,Café,Breakfast Spot,Sandwich Place,Italian Restaurant,Japanese Restaurant,Restaurant,Shopping Mall
8,Downtown Toronto,1,Restaurant,Café,Coffee Shop,American Restaurant,Steakhouse,Hotel,Asian Restaurant,Japanese Restaurant,Gym,Thai Restaurant
9,West Toronto,1,Bakery,Park,Pet Store,Middle Eastern Restaurant,Café,Bar,Bank,Pharmacy,Gym / Fitness Center,Grocery Store
10,Downtown Toronto,1,Harbor / Marina,Music Venue,Café,Park,Curling Ice,Dumpling Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


# As you can see Coffee Shops and Cafe's are very common in this cluster, hence this cluster of Neighborhood is suitable for all the Coffee Lovers out there.